In [91]:
# Summary of the algorithm
- We start with input sequences from a domain (e.g. English sentences)
    and correspding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    Is uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.

SyntaxError: invalid syntax (<ipython-input-91-1c8c0c93471c>, line 2)

In [1]:
from __future__ import print_function

from keras.models import Model
from keras.models import load_model
from keras.layers import Input, LSTM, Dense
import numpy as np
import json

Using TensorFlow backend.


In [5]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = '/Users/guru/MyResearch/sg/snli/plain/snli_dev.txt'

In [7]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path).read().split('\n')

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text = line
    target_text = line
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
            
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

#np.zeros 0を要素とする配列(shape, dtype = float, order = ‘C’)　３次元
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')


for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

Number of samples: 10000
Number of unique input tokens: 74
Number of unique output tokens: 76
Max sequence length for inputs: 300
Max sequence length for outputs: 302


In [8]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [5]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('model/train/s2s_1117_snli.h5')

Train on 15919 samples, validate on 3980 samples
Epoch 1/10
   64/15919 [..............................] - ETA: 55:00 - loss: 0.9084

KeyboardInterrupt: 

In [5]:
# choose input data version
input_texts_inf = []
lines = open('predict.txt').read().split('\n')

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text_inf = line
    input_texts_inf.append(input_text_inf)
            
print('Number of samples:', len(input_texts_inf))


#np.zeros 0を要素とする配列(shape, dtype = float, order = ‘C’)　３次元
encoder_input_data_inf = np.zeros(
    (len(input_texts_inf), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')


for i, input_text_inf in enumerate(input_texts_inf):
    for t, char in enumerate(input_text_inf):
        encoder_input_data_inf[i, t, input_token_index[char]] = 1.
       

Number of samples: 1


In [9]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
model = load_model('s2s.h5')

encoder_model = load_model('encoder.h5')
#encoder_model = Model(encoder_inputs, encoder_states)
#encoder_model.save('model/encoder/s2s_1117_snli_ccg_10.h5')


decoder_model = load_model('decoder.h5')
#decoder_state_input_h = Input(shape=(latent_dim,))
#decoder_state_input_c = Input(shape=(latent_dim,))
#decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
#decoder_outputs, state_h, state_c = decoder_lstm(
#    decoder_inputs, initial_state=decoder_states_inputs)
#decoder_states = [state_h, state_c]
#decoder_outputs = decoder_dense(decoder_outputs)
#decoder_model = Model(
#    [decoder_inputs] + decoder_states_inputs,
#    [decoder_outputs] + decoder_states)
#decoder_model.save('model/decoder/s2s_1117_snli_ccg_10.h5')

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())

reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

/Users/guru/.pyenv/versions/3.5.1/lib/python3.5/site-packages/keras/models.py:251: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [10]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [11]:
for seq_index in range(100):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    #input_seq = encoder_input_data[seq_index: seq_index + 1]
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Two women are embracing while holding to go packages.
Decoded sentence: Two women are smarged on the street.

-
Input sentence: The sisters are hugging goodbye while holding to go packages after just eating lunch.
Decoded sentence: The young man is practicing for a field with a little boy behind him.

-
Input sentence: Two women are embracing while holding to go packages.
Decoded sentence: Two women are smarged on the street.

-
Input sentence: Two woman are holding packages.
Decoded sentence: Two women are eximing something in a park on the grassed buichcars tear.

-
Input sentence: Two women are embracing while holding to go packages.
Decoded sentence: Two women are smarged on the street.

-
Input sentence: The men are fighting outside a deli.
Decoded sentence: The men are playing a group of people.

-
Input sentence: Two young children in blue jerseys, one with the number 9 and one with the number 2 are standing on wooden steps in a bathroom and washing their hands